# Introduction

In this notebook, we analyse a sample of PBMCs. Here, we cover normalization, batch effect correction, selection of highly variable genes, computing cell cycle scores, conduct an initial clustering and visualisation of the data.


Install all packages for the tutorial.

In [3]:
!pip install scanpy==1.6.1 umap-learn==0.4.6 anndata==0.7.5 numpy==1.19.5 scipy==1.4.1 pandas matplotlib scrublet seaborn python-igraph==0.8.3 louvain==0.7.0 leidenalg==0.8.3

     |████████████████████████████████| 10.2MB 4.4MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 122kB 49.3MB/s 
     |████████████████████████████████| 3.2MB 39.8MB/s 
     |████████████████████████████████| 2.2MB 49.5MB/s 
     |████████████████████████████████| 2.4MB 46.7MB/s 
     |████████████████████████████████| 655kB 44.3MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.4.6-cp37-none-any.whl size=67950 sha256=73d5cb1d3c6d934d02650d176b11047ea7b19bdb89855adbfc18070a7ff26e46
  Stored in directory: /root/.cache/pip/wheels/7d/1d/03/34aade9a9b97acddb8e93654eb856dadbf0964406eef8b96e2
  Created wheel for sinfo: filename=sinfo-0.3.1-cp37-none-any.whl size=7012 sha256=bb127fa685485f839524349037891d37cc1fef1249ba1d798b14834188c16250
  Stored in directory: /root/.cache/pip/wheels/11/f0/23/347d6d8e59787c2bc272162d18223dc3

Load all required packages.

In [4]:
import scanpy as sc
import anndata as ann
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors

import os 
#doublet detection
import scrublet as scr


#pretty plotting
import seaborn as sb



In [5]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_header()


scanpy==1.6.1 anndata==0.7.5 umap==0.5.1 numpy==1.19.5 scipy==1.4.1 pandas==1.1.5 scikit-learn==0.22.2.post1 statsmodels==0.10.2 python-igraph==0.8.3 louvain==0.7.0 leidenalg==0.8.3


Of note, this notebook was created as part of a workshop, so we use extra large legend texts in all seaborn plots. You can set the context as well to 'talk' or 'paper'.

In [6]:
sb.set_context(context='poster')


# Set project file paths

Let us set up the connection with Google Drive.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We set up the file paths to the respective directories.

In [8]:
file_path = '/content/drive/My Drive/' #this is the file path to your google drive (main directory)

In [9]:
import os 

In [ ]:
os.listdir(file_path)

File path to the raw data. They are usually stored at a different location than the rest of the project.

In [11]:
file_path_raw = file_path + '3k_PBMC/'

The data directory contains all processed data and `anndata` files. 

In [12]:
data_dir = file_path + 'PBMC_Colabs/data/' 

The tables directory contains all tabular data output, e.g. in `.csv` or `.xls` file format. That applies to differential expression test results or overview tables such as the number of cells per cell type.

In [13]:
table_dir = file_path + 'PBMC_Colabs/tables/'

The default figure path is a POSIX path calles 'figures'. If you don't change the default figure directory, scanpy creates a subdirectory where this notebook is located.  

In [14]:
sc.settings.figdir = file_path + 'PBMC_Colabs/figures/'

**Comment:** When you repeat certain analyses, it might be helpful to set a `date` variable and add it to every figure and table (see `datetime` Python package).

In [ ]:
import datetime

today = datetime.date.today().strftime('%y%m%d') #creates a YYMMDD string of today's date

# Read data

The post-QC dataset, which consists of 3k PBMCs (Human) provided by 10X Genomics.

In [13]:
adata = sc.read(data_dir + 'data_postQC.h5ad')

# Normalization and Batch effect correction

## Normalization

So far, our dataset is a count matrix. Here, every count corresponds to an mRNA molecule captured in the scRNA-seq experiment. As not all mRNA molecules in a cell are captured, there is a variability in the total number of counts detected between cells that results from both the number of molecules that were in the cells, and the sampling. As we cannot assume that all cells contain an equal number of molecules (cell sizes can differ substantially), we have to estimate the number of molecules that were initially in the cells. In fact, we don't estimate the exact number of molecules, but instead estimate cell-specific factors that should be proportional to the true number of molecules. These are called size factors. Normalized expression values are calculated by dividing the measured counts by the size factor for the cell.

Several methods for normalization for scRNA-seq data have been proposed. Ideally, we want to apply the `scran` library size normalization followed by log-transformation. However, `scran` is again an R package, and we use the simpler normalisation to logCPM, `i.e.` we divide by the library size followed by the log-transformation. In addition, we save the count matrix to `layers` as 'counts'.

**Comment:** For `scran`, we have not (comprehensively) tested whether normalisation per sample or all samples jointly gives more accurate results. When we normalised per sample, observed many more differentially expressed genes across conditions (in the range of thousands). We think that normalisation per sample preserves a systematic bias while joint normalisation removes batch effects within the same cluster partially, if a cluster contains cell from several batches. No such effect of the normalisation can be observed when samples do not overlap at all. For the time being, we perform joint normalisation of all samples. 

In [39]:
adata.layers['counts'] = adata.X.copy()

We use the logCPM normalisation, with minor adaptations: After normalization, each observation (cell) has a total count equal to the median of total counts for observations (cells) before normalization. Then, we scale all values with log(x+1).

In [40]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

normalizing counts per cell
    finished (0:00:00)


**Tasks:** Plot library size vs number of expressed genes.

**Questions:** How does the library size relate to number of expressed genes? Can we suspect cell type specific differences in the data? 

## Batch correction - general remarks

This dataset consists of a single batch, thus, batch effect correction is not an issue here. If you handle several batches, you may observe differences across samples, for instance, in the library size per dataset. Such differences may contribute to the batch effect. [Büttner et al., Nat Meth (2019)](https://www.nature.com/articles/s41592-018-0254-1) compared the performance of several batch correction methods. For low-to-medium complexity datasets, ComBat performed best among the tested tools. ComBat is also available in `scanpy` (see `sc.pp.combat`). 

For high complexity data, especially when you encounter changes in cell type composition, consider to use a data integration method of your choice. We distinguish three different types, *i.e.* if the methods create a corrected data matrix (in feature space), an embedding or a knn-graph. Examples are:
1. feature space: MNN (`scanpy.external.pp.mnn_correct`), scanorama (integrates with scanpy), Seurat v3 (R based)
2. embedding: scVI (Python based), Harmony (R based), scanorama
3. knn-graph: conos (R based), BBKNN (`scanpy.external.pp.bbknn`)

Several benchmarking studies aimed to determine best performing methods. In simple cases, Seurat v3 and Harmony performed best [Tran et al., Genome Biology (2020)](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1850-9) and [Chazarra-Gil et al., biorxiv (2020)](https://www.biorxiv.org/content/10.1101/2020.05.22.111211v2). More complex scenarios have been benchmarked in [Luecken et al., biorxiv (2020)](https://www.biorxiv.org/content/10.1101/2020.05.22.111161v2), where BBKNN, Scanorama, and scVI performed well. Furthermore, Scanorama had high scores in the preservation of biological signals, while BBKNN tended to overcorrect. Moreover, Luecken et al. tested different pre-processing schemes. In general, **selecting highly variable genes prior to batch correction improved the batch effect correction result**. We continue with the selection of highly variable genes. 


## Feature selection (Highly variable genes)

We extract highly variable genes (HVGs) to further reduce the dimensionality of the dataset and include only the most informative genes. Genes that vary substantially across the dataset are informative of the underlying biological variation in the data. As we only want to capture biological variation in these genes, we select highly variable genes after normalization and batch correction. HVGs are used for clustering, trajectory inference, and dimensionality reduction/visualization, while the full data set is used for computing marker genes, differential testing, cell cycle scoring, and visualizing expression values on the data.

Here we use a standard technique for the extraction of highly variable genes from the 10X genomics preprocessing software *CellRanger*. Typically between 1000 and 5000 genes are selected. Here, we extract the top 4000 most variable genes for further processing. If particular genes of importance are known, one could assess how many highly variable genes are necessary to include all, or the majority, of these.

Compute highly variable genes and visualize.

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=2000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
rcParams['figure.figsize']=(10,5)
sc.pl.highly_variable_genes(adata)

The plots show how the data was normalized to select highly variable genes irrespective of the mean expression of the genes. This is achieved by using the index of dispersion which divides by mean expression, and subsequently binning the data by mean expression and selecting the most variable genes within each bin.

Highly variable gene information is stored automatically in the `adata.var['highly_variable']` field. The dataset now contains:

* count data as layer 'counts' in adata
* log-scran normalized data in adata.X
* highly variable gene annotations in `adata.var['highly_variable']`


## Cell cycle scoring

Known sources of technical variation in the data have been investigated and corrected for (e.g. batch, count depth). A known source of biological variation that can explain the data is the cell cycle. Here, gene lists from [Macosko et al., Cell 161 (2015)](https://www.sciencedirect.com/science/article/pii/S0092867415005498) is used to score the cell cycle effect in the data and classify cells by cell cycle phase. The file can be found on the [scIB github repository](https://github.com/theislab/scib/tree/master/scIB/resources/).

Please note, that the gene list was generated for human HeLa cells.

In [ ]:
s_genes_file = data_dir + 's_genes_tirosh_hm.txt'
g2m_genes_file = data_dir + 'g2m_genes_tirosh_hm.txt'

In [ ]:
s_genes = pd.read_table(s_genes_file, header = None).values.flatten()
g2m_genes = pd.read_table(g2m_genes_file, header = None).values.flatten()

In [ ]:
s_genes_hvg = adata.var_names[np.in1d(adata.var_names, s_genes)]
g2m_genes_hvg = adata.var_names[np.in1d(adata.var_names, g2m_genes)]

Compute cell cycle score.

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes_hvg, g2m_genes=g2m_genes_hvg)

In [ ]:
print(len(s_genes_hvg))
print(len(g2m_genes_hvg))

In [ ]:
adata.obs['phase'].value_counts()

**COMMENT:** End of third session. 

# Visualization

## low dimensional embeddings

Visualizing scRNA-seq data is the process of projecting a high-dimensional matrix of cells and genes into a few coordinates such that every cell is meaningfully represented in a two-dimensional graph. However, the visualization of scRNA-seq data is an active area of research and each method defines 'meaningful' in its own way. 

Overall t-SNE visualizations have been very popular in the community, however the recent UMAP algorithm has been shown to better represent the topology of the data.

We will look at several visualizations to decide which visualization best represents the aspect of the data.

Compute the following embeddings: PCA, t-SNE, UMAP, diffusion map and force-directed graph. Please compute PCA first and compute nearest neighbors next. All other embeddings rely on this information. Visualize the embeddings and color by the total number of counts.

In [ ]:
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')

To determine the number of informative principal components, let us review the variance contribution of each component. 

In [ ]:
sc.pl.pca_variance_ratio(adata)

Using the elbow method, we select the first **XX** PCs as informative.

It must be noted that the elbow method is not a strict scientific criterion to select the correct number of PCs. For instance, the variance of the gene expression from rare cell types might only be covered by the latter PCs. Therefore, consider to try out different numbers of PCs and examine the differences in the resulting low-dimensional embeddings.

**Task:** Decide on the number of informative principal components.

In [ ]:
sc.pp.pca(adata, n_comps= ,#number of informative components  
          use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata)
sc.tl.tsne(adata) #Note n_jobs works for MulticoreTSNE, but not regular implementation)
sc.tl.umap(adata)
sc.tl.diffmap(adata)
sc.tl.draw_graph(adata)

In [ ]:
rcParams['figure.figsize']=(20,10)
fig_ind=np.arange(231, 237)
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.6)

p10 = sc.pl.pca_scatter(adata, color='n_counts', ax=fig.add_subplot(fig_ind[0]), show=False)
p11 = sc.pl.tsne(adata, color='n_counts', ax=fig.add_subplot(fig_ind[1]), show=False)
p12 = sc.pl.umap(adata, color='n_counts', ax=fig.add_subplot(fig_ind[2]), show=False)
p13 = sc.pl.diffmap(adata, color='n_counts', components=['1,2'], ax=fig.add_subplot(fig_ind[3]),show=False)
p14 = sc.pl.diffmap(adata, color='n_counts', components=['1,3'], ax=fig.add_subplot(fig_ind[4]), show=False)
p15 = sc.pl.draw_graph(adata, color='n_counts', ax=fig.add_subplot(fig_ind[5]), show=False)

plt.show()

**PCA**:

* Unsurprisingly, the first principle component captures variation in count depth between cells, and is thus only marginally informative
* The plot shows a weak clustering of the data in two dimensions

**t-SNE**:

* Shows several distinct clusters with clear subcluster structure
* Connections between clusters are difficult to interpret visually

**UMAP**:

* Data points are spread out on the plot showing several clusters
* Connections between clusters also not visible (and not expected for PBMC data)

**Diffusion Maps**:

* Shows regions of higher density and a few disconnected cells in between
* Trajectories not visible (and not expected for PBMC data)
* Each diffusion component extracts heterogeneity in a different part of the data

**Graph**:

* Shows several clusters with substructure
* Lack of trajectories as expected

The strengths and weaknesses of the visualizations can readily be identified in the above plots. While t-SNE exaggerates differences, diffusion maps exaggerate transitions. Overall UMAP and force-directed graph drawings show the best compromise of the two aspects, however UMAP is much faster to compute. UMAP has furthermore been shown to more accurately display the structure in the data.

Next, let us compute an embedding density. We use the UMAP representation as basis.

In [ ]:
sc.tl.embedding_density(adata, basis='umap')

In [ ]:
rcParams['figure.figsize']=(7,7)
sc.pl.embedding_density(adata, basis='umap')

## Visualise cell cycle score

**Task:** Check whether MKI67 is present in the dataset and visualise it.

In [ ]:
#hint 'MKI67' has to be in the adata.var_names 

In [ ]:
rcParams['figure.figsize']=(5,5)
#plots

**Task:** Plot a low-dimensional embedding and show the cell cycle scores and the predicted phase.

In [ ]:
rcParams['figure.figsize']=(5,5)
#plots

Apparently, all cells are assigned to the same proliferative cell cycle phase (G2M), but inspecting the interface marker MKI67 shows little evidence for proliferation. If the phase score looks suspicious, one has to adjust the threshold for assigning a certain phase. By default, a cell is G1, if both S and G2M score are negative. Otherwise, a cell is assigned to the phase where it has the highest score. Potentially, we would need to adapt the cutoffs of the classification.

At this point of the analysis, we have reached an important milestone as we finished the pre-processing and enter the downstream analysis part. **Ideally, we don't have to revisit this part again.**

In [ ]:
adata.write(data_dir + 'data_processed.h5ad')

**Comment:** End of third session.

## Visualise gene expression

In this session, we focus on the various plotting functions in `scanpy` to visualise gene expression patterns per cell and for groups of cells. On the one hand, using various ways of visualising data helps to find patterns and identify cell types. On the other hand, we aim to show how to prettify plots in `scanpy`. 

We will look at visualising gene expression:
* on low dimensional embedding plots
* as heatmap
* as tracksplot
* as matrixplot
* as (stacked) violin plot
* as dotplot


Please note that we leave the "stringent" data analysis workflow. When you start working with `scanpy` in your own analysis projects, you might single out a few plotting functions, which you find the most useful to quickly explore the data, and only reconsider other means to visualise data only upon finalising a project. 

In order to display groups of cells, we will use a clustering approach called "leiden clustering", which will be explained in detail on day 3.



Let us visualise gene expression on low dimensional embeddings. We use some marker genes for PBMCs (e.g. CD79A, CD3D, MS4A1, FCER1A).  

In [ ]:
rcParams['figure.figsize'] = 3,3
sc.pl.umap(adata, color=['CD79A', 'MS4A1', 'CD3D', 'FCER1A'], 
           ncols=2, #number of columns to use 
           hspace=0.5, #space between 2 rows
           wspace=0.3, #space between 2 columns
          )

Let us prettify the plots: The maximum value plotted can be adjusted using `vmax` (similarly vmin can be used for the minimum value). In this case we use `p99`, which means to use as max value the 99 percentile (less biased by outliers). The max value can be a number or a list of numbers if the `vmax` wants to be set for multiple plots individually.

Also, we are using `frameon=False` to remove the boxes around the plots and `s=50` to set the dot size. Lastly, we change the color map to a Red scale.

In [ ]:
rcParams['figure.figsize'] = 3,3
sc.pl.umap(adata, color=['CD79A', 'MS4A1', 'CD3D', 'FCER1A'], 
           hspace=0.5, #space between 2 rows
           wspace=0.3, #space between 2 columns
           s=50, #dot size
           frameon=False, ncols=2, vmax='p99',
           color_map='Reds'
          )

Let us scale the data and save it as `layer`.

In [ ]:
# scale and store results in layer
adata.layers['scaled'] = sc.pp.scale(adata, copy=True).X

Compute low resolution clustering.

In [ ]:
sc.tl.leiden(adata, key_added='clusters', resolution=0.5)

Visualise clustering. The functions for scatterplots have many options that allow fine tuning of the images. For example, we can look at the clustering as follows:

In [ ]:
rcParams['figure.figsize'] = 5, 5
sc.pl.umap(adata, color='clusters', #plot categorical values
           add_outline=True, #black outline on all cells
           legend_loc='on data', #plot cluster ID on data
           legend_fontsize=12, 
           legend_fontoutline=2,
           frameon=False,
           title='clustering of cells', 
           palette='Set3' #use a colorbrewer scheme for categorical data
          )

The full set of options can be found in the documentation of the plotting functions.

Create a violin plot with and without internal dots.

In [ ]:
sc.pl.violin(adata, keys=['CD3D', 'MS4A1', 'FCER1A'],
              groupby='clusters', 
              dendrogram=True, figsize=(5,4))
sc.pl.violin(adata, keys=['CD3D', 'MS4A1', 'FCER1A'],
              groupby='clusters',  stripplot=False,
              dendrogram=True, figsize=(5,4))

Create a heatmap.

In [ ]:
sc.pl.heatmap(adata, var_names=['CD3D', 'MS4A1', 'FCER1A'], 
              figsize=(7,5), #set figure shape
              groupby='clusters', cmap='viridis', dendrogram=True)

Use the `swap_axes=True` command to create a horizontal heatmap.

In [ ]:
sc.pl.heatmap(adata, var_names=['CD3D', 'MS4A1', 'FCER1A'], 
              groupby='clusters', 
              cmap='viridis', #color map
              dendrogram=True, swap_axes=True, figsize=(7,5))

Show the scaled version of the data and use a diverging color map.

In [ ]:
sc.pl.heatmap(adata, var_names=['CD3D', 'MS4A1', 'FCER1A'], layer='scaled',
              groupby='clusters', 
              vmin=-2, vmax=2, #fix minimum and maximum
              cmap='RdBu_r',  #color map
              dendrogram=True, swap_axes=True, figsize=(7,5))

Use a tracksplot instead of a heatmap.

In [ ]:
sc.pl.tracksplot(adata, var_names=['CD3D', 'MS4A1', 'FCER1A'],
              groupby='clusters', 
              dendrogram=True, swap_axes=True, figsize=(10,5))

Create a dotplot, a matrixplot and a stacked violin plot by replacing the corresponding keyword. 

In [ ]:
sc.pl.dotplot(adata, var_names=['CD3D', 'MS4A1', 'FCER1A'],
              groupby='clusters', 
              dendrogram=True, figsize=(5,4))

sc.pl.stacked_violin(adata, var_names=['CD3D', 'MS4A1', 'FCER1A'],
              groupby='clusters', 
              dendrogram=True, figsize=(5,4))

sc.pl.matrixplot(adata, var_names=['CD3D', 'MS4A1', 'FCER1A'],
              groupby='clusters', 
              dendrogram=True, figsize=(5,5))

Advanced plotting: Add cluster size as barchart to the top of the plot.

In [ ]:
vp = sc.pl.stacked_violin(adata,var_names=['CD3D', 'MS4A1', 'FCER1A'], swap_axes=True,
              groupby='clusters', return_fig=True)
vp.add_totals().style(ylim=(0,5)).show()

More inspiration can be found in the [scanpy plotting tutorial](https://scanpy-tutorials.readthedocs.io/en/latest/plotting/core.html) and the [scanpy plotting documentation](https://scanpy.readthedocs.io/en/stable/api/scanpy.plotting.html). 

**Tasks:** Choose one of the plots and customize it to your liking. 

Examples:
* Take your favourite embedding and display all genes of interest on the same color scale (0-1). 
* Choose your own color scale, please get inspired from the [list of colors](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html)
* Plot only two of the clusters.
* Remove the dendrogram.
* Change the figure size and shape.
* Save the plot to file (formats: PNG, PDF) - check, if the margins are correct.


Here's a list of marker genes from literature for the subpopulations of PBMCs.

In [ ]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

**Comment:** End of fourth session and day 2.

Save data to file.

In [ ]:
adata.write(data_dir + 'data_processed-vis.h5ad')